<td>
   <a target="_blank" href="https://labelbox.com" ><img src="https://labelbox.com/blog/content/images/2021/02/logo-v4.svg" width=256/></a>
</td>

<td>
<a href="https://colab.research.google.com/github/Labelbox/labelbox-python/blob/master/examples/basics/user_management.ipynb" target="_blank"><img
src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"></a>
</td>

<td>
<a href="https://github.com/Labelbox/labelbox-python/tree/master/examples/basics/user_management.ipynb" target="_blank"><img
src="https://img.shields.io/badge/GitHub-100000?logo=github&logoColor=white" alt="GitHub"></a>
</td>

# User Management
* This notebook covers the following:
    * create invites
    * query for remaining allowed invites to an organization
    * set and update organization roles
    * assign users to projects
        * set / update / revoke project role
    * delete users from org

In [1]:
!pip install labelbox

In [2]:
from labelbox import Project, Dataset, Client, User
from labelbox.schema.role import ProjectRole
from labelbox.schema.media_type import MediaType
import os

* You have to specifically enable experimental features to use this functionality. Notice the 
`enable_experimental = True`
    * enables users to send invites and checking the number of seats available via the sdk

# API Key and Client
Provide a valid api key below in order to properly connect to the Labelbox Client.

In [4]:
# Add your api key
API_KEY = None
client = Client(api_key=API_KEY, enable_experimental=True)
organization = client.get_organization()

In [5]:
# Please provide a dummy email here:
# Preferrably one you can access. If you have a google account you can do email+1@<domain>.com
DUMMY_EMAIL = "SET THIS"
# This should be set to an account that you wan't to change the permissions for.
# You could invite a new user, accept the invite and use that account if you don't want to effect any active users
DUMMY_USER_ACCOUNT_ID = "ckneh4n8c9qvq0706uwwg5i16"

### Roles
* When inviting a new user to an organization, there are various roles to select from.
* All available roles to your org can be accessed via `client.get_roles()`

In [6]:
roles = client.get_roles()
for name, role in roles.items():
    print(role.name, ":", role.uid)

LABELER : cjlvi914y1aa20714372uvzjv
REVIEWER : cjlvi919b1aa50714k75euii5
TEAM_MANAGER : cjlvi919q1aa80714b7z3xuku
ADMIN : cjlvi91a41aab0714677xp87h
NONE : cjmb6xy80f5vz0780u3mw2rj4


* Above we printed out all of the roles available to the current org.
* Notice the `NONE`. That is for project level roles

### Create
* Users are created by sending an invite
* An email will be sent to them and they will be asked to join your organization

#### Organization Level Permissions
* Invite a new labeler with labeling permissions on all projects

In [7]:
# First make sure that you have enough seats:
organization.invite_limit()

InviteLimit(remaining=2, used=3, limit=5)

In [8]:
invite = organization.invite_user(DUMMY_EMAIL, roles["LABELER"])

In [9]:
print(invite.created_at)
print(invite.organization_role_name)
print(invite.email)

2021-04-20 22:11:55.897000+00:00
Labeler
< SET THIS >


#### Project Level Permissions
* Invite a new labeler with labeling permissions specific to a set of projects
* Here we set organization level permissions to Roles.NONE to indicate that the user only has project level permissions

In [10]:
project = client.create_project(name="test_user_management", media_type=MediaType.Image)
project_role = ProjectRole(project=project, role=roles["REVIEWER"])
invite = organization.invite_user(DUMMY_EMAIL,
                                  roles["NONE"],
                                  project_roles=[project_role])

### Read
* Outstanding invites cannot be queried for at this time. This information can be found in the members tab of the web app.
* You are able to query for members once they have joined.

In [11]:
users = list(organization.users())
print(users[0])

<User {'created_at': datetime.datetime(2021, 1, 20, 1, 2, 31, tzinfo=datetime.timezone.utc), 'email': 'msokoloff@labelbox.com', 'intercom_hash': '050b8b292bd66abf88ea9a279c6ee80a530e0f9cf98a06e1d5372ef13697f46b', 'is_external_user': False, 'is_viewer': True, 'nickname': 'msokoloff', 'name': 'Matt Sokoloff', 'picture': 'https://lh6.googleusercontent.com/-gO94Mcw4PGs/AAAAAAAAAAI/AAAAAAAAAAA/AMZuuclv_5P5WICzV1aImhxGADj_OS5c7w/s96-c/photo.jpg', 'uid': 'ckk4q1vgwc0vp0704xhx7m4vk', 'updated_at': datetime.datetime(2021, 4, 1, 15, 48, 27, tzinfo=datetime.timezone.utc)}>


### Update
* There is no update on invites. Instead you must delete and resend them
* You can update User roles

In [12]:
user = client._get_single(User, DUMMY_USER_ACCOUNT_ID)

# Give the user organization level permissions
user.update_org_role(roles["LABELER"])
print(user.org_role())
# Restore project level permissions
user.update_org_role(roles["NONE"])
print(user.org_role())
# Make the user a labeler for the current project
user.upsert_project_role(project, roles["LABELER"])
print(user.org_role())

<OrgRole {'name': 'Labeler', 'uid': 'cjlvi914y1aa20714372uvzjv'}>
<OrgRole {'name': 'None', 'uid': 'cjmb6xy80f5vz0780u3mw2rj4'}>
<OrgRole {'name': 'None', 'uid': 'cjmb6xy80f5vz0780u3mw2rj4'}>


In [13]:
# Remove the user from a project (Same as setting the project role to `roles.NONE`)
user.remove_from_project(project)

### Delete

* Invites can only be deleted from the ui at this time. 
* Deleting invites can be done in the members tab of the web app.

* Delete the User
* Make sure you want to remove the user from the org:
* `>>> organization.remove_user(user)`

### Cleanup
* We created an extra project. Let's delete it

In [14]:
project.delete()